# **Question 3: Form a portfolio, data selection, and necessary transformation**

## Part 1 - Creating portfolio

### We aim to create a portfolio consisting of stocks from the sectors that have the most weight in the S&P 500 (Healthcare, Tech, Consumer Discretionary, and Consumer Staples). We are going to weight the portfolio by their market caps. The goal is to then stress macroeconomic indicators like CPI, interest rates, and unemployment as MEVs later and see if they have a substantial effect on how our portfolio performs.

### Breakdown of sectors:

- **Healthcare**
  - “Defensive/Non-cyclical” – perform well during economic downturns due to consistent demand for healthcare and services
  - Relevant Macro Factors: less sensitive to changes in real GDP growth or UE, but affected by inflation (drug pricing pressure) and interest rates (affect R&D borrowing costs)

- **Financials**
  - “Cyclical” – performance tied to business cycle/economic growth; benefits from higher interest rates, vulnerable during downturns
  - Relevant Macro Factors: real GDP growth, nominal GDP growth, interest rates have impacts on bank earnings

- **Consumer Staples/Food**
  - “Defensive” – Consumer staples are products that are essential regardless of economic benefits (food, beverages, household goods) → Perform well during recessions and periods of high inflation (companies have pricing power/consistent demand)
  - Industrials and Food products are “defensive” but sensitive to commodity price fluctuations and inflation; can benefit from rising prices but suffer from rising input costs
  - Relevant Macro Factors (CS) : CPI inflation (companies pass inflation onto the consumer) and real disposable income growth (spending power)
  - Relevant Macro Factors (Food): real GDP growth, CPI inflation, commodity prices

In [50]:
import pandas as pd
import yfinance as yf
import numpy as np
import cvxpy as cp
from statsmodels.tsa.stattools import adfuller

In [51]:
from portfolio import *

In [52]:
tickers = [
    'AAPL', 'MSFT',                                   # Information Technology
    'BIIB', 'JNJ', 'LLY', 'MRK', 'PFE',               # Health Care
    'AMZN', 'NKE',                                    # Consumer Discretionary
    'JPM', 'BAC', 'C', 'MS',                          # Financials
    'GOOGL',                                          # Communication Services
    'HON', 'UNP',                                     # Industrials
    'PG', 'KO', 'WMT', 'CL', 'TSN',                   # Consumer Staples
    'XOM', 'CVX',                                     # Energy
    'NEE',                                            # Utilities
    'PLD', 'AMT',                                     # Real Estate
    'LIN'                                             # Materials
]

equal_weight = 1/len(tickers)
weights = {ticker:equal_weight for ticker in tickers}
prices = yf.download(tickers, start='1976-04-01', end='2023-10-01', interval='1mo', progress=False)['Adj Close']

In [53]:
prices.index = pd.to_datetime(prices.index)
prices.index = prices.index.strftime('%Y-%m')
prices.dropna(inplace=True)
prices.head()

Ticker,AAPL,AMT,AMZN,BAC,BIIB,C,CL,CVX,GOOGL,HON,...,MSFT,NEE,NKE,PFE,PG,PLD,TSN,UNP,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2004-09,0.583704,11.720683,2.0430,27.297626,61.169998,278.599030,14.123342,25.307062,3.235232,21.777493,...,17.250330,4.690567,7.723265,12.824574,30.943277,18.852253,11.653690,9.766551,11.790459,24.749008
2004-10,0.789319,13.125639,1.7065,28.502579,58.160000,280.177765,13.948284,25.033434,4.758987,20.453604,...,17.449972,4.730388,7.979590,12.133056,29.262312,19.096687,10.547975,10.549772,11.950028,25.204746
2004-11,1.009996,13.843385,1.9840,29.444399,58.680000,285.172852,14.453048,25.759979,4.542805,21.455631,...,16.726269,4.828566,8.308350,11.638510,30.722263,20.580612,11.922843,10.628516,11.537807,26.244282
2004-12,0.970079,14.049549,2.2145,29.902565,66.610001,307.030487,16.077793,24.962437,4.812658,21.617928,...,18.579533,5.180113,8.900124,11.337541,31.641407,20.807280,13.417189,11.266830,11.706242,26.391439
2005-01,1.158370,13.835753,2.1610,29.797827,64.959999,312.574829,16.511478,25.860910,4.883303,21.965914,...,18.273582,5.311087,8.513955,10.186500,30.578627,19.383289,12.520279,10.032346,11.641443,26.566494


In [54]:
len(prices) / 12

19.083333333333332

## Part 2 - Describing handling of missing data

### As most of these companies weren't created dating back to the late 1900s, we chose to drop those leading years. To handle any missing data (as gaps or singular missing cells) within the price series, we plan to use forward/backward fill or interpolate, depending on the volatility of the stock if we run into that situation. We felt that deleting these previous years was the best way of going about this because filling in the values dating that far back would introduce significant bias into our analysis, no matter how we filled the data. It seemed like the best option to get rid of them all together, and after doing so we still have 19 years worth of data to work with which is more than enough. Specifically, we have 228 months worth of observations.

## Part 3 - Describing external data sources

### The only other data source outside of the given data files used to source stock prices was the yahoo finance (yfinance) API.

## Part 4 - Stationarity tests on MEVs

In [55]:
MEVs = pd.read_csv('MEV_actual.csv')
quarter_to_month = {'Q1': '03', 'Q2': '06', 'Q3': '09', 'Q4': '12'}

def convert_to_yyyy_mm(quarter_str):
    year, quarter = quarter_str.split()
    month = quarter_to_month[quarter]
    return f"{year}-{month}"
    
MEVs['Date'] = MEVs['Date'].apply(convert_to_yyyy_mm)
MEVs.set_index('Date', inplace=True)
MEVs.head()

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Mortgage rate,Prime rate,House Price Index (Level),Commercial Real Estate Price Index (Level)
Date,,,,,,,,,,,,,
1976-03,9.3,14.0,5.0,9.6,7.7,4.7,4.9,7.4,7.6,8.9,6.8,22.9,50.9
1976-06,3.0,7.2,2.3,5.8,7.6,3.6,5.2,7.4,7.6,8.8,6.9,23.6,51.8
1976-09,2.2,7.6,3.2,9.6,7.7,6.5,5.2,7.3,7.6,9.0,7.1,24.2,52.6
1976-12,2.9,10.5,2.6,9.2,7.8,5.9,4.7,6.5,7.1,8.8,6.5,25.2,53.4
1977-03,4.8,11.7,0.9,8.4,7.5,7.5,4.6,6.8,7.2,8.7,6.3,26.2,55.0


In [56]:
def make_stationary(series, significance_level=0.05):
    """
    Differentiates a pandas Series until it becomes stationary based on the Augmented Dickey-Fuller test.
    
    Parameters:
    - series: pd.Series - The time series data to test for stationarity.
    - significance_level: float - The p-value threshold to consider the series stationary (default is 0.05).
    
    Returns:
    - num_diffs: int - The number of differences needed to achieve stationarity.
    """
    num_diffs = 0
    diff_series = series.copy()
    
    while True:

        adf_test = adfuller(diff_series.dropna())
        p_value = adf_test[1]
        
        if p_value < significance_level:
            return num_diffs, diff_series
        
        diff_series = diff_series.diff().dropna()
        num_diffs += 1

In [57]:
diffs_needed = pd.DataFrame(index=MEVs.columns, columns=['Differences'])
for MEV in MEVs.columns:
    stationary = make_stationary(MEVs[MEV])
    diffs_needed.loc[MEV] = stationary[0]
    if stationary[0] != 0:
        MEVs[MEV] = stationary[1]
MEVs = MEVs.dropna()
MEVs.head()

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Mortgage rate,Prime rate,House Price Index (Level),Commercial Real Estate Price Index (Level)
Date,,,,,,,,,,,,,
1976-06,3.0,7.2,2.3,-3.8,7.6,-1.1,0.3,0.0,0.0,-0.1,0.1,0.7,0.9
1976-09,2.2,7.6,3.2,3.8,7.7,2.9,0.0,-0.1,0.0,0.2,0.2,0.6,0.8
1976-12,2.9,10.5,2.6,-0.4,7.8,-0.6,-0.5,-0.8,-0.5,-0.2,-0.6,1.0,0.8
1977-03,4.8,11.7,0.9,-0.8,7.5,1.6,-0.1,0.3,0.1,-0.1,-0.2,1.0,1.6
1977-06,8.0,14.2,3.8,2.7,7.1,-0.3,0.2,0.0,0.1,0.1,0.2,1.3,1.0


In [58]:
diffs_needed[diffs_needed['Differences']!=0]

,Differences
Nominal disposable income growth,1
CPI inflation rate,1
3-month Treasury rate,1
5-year Treasury yield,1
10-year Treasury yield,1
Mortgage rate,1
Prime rate,1
House Price Index (Level),1
Commercial Real Estate Price Index (Level),1


### As standard practice for stationarity testing, we used the Augemented Dickey-Fuller test with a default significance level of $\alpha=5\%$. In the case where the series wasn't stationary initially, the function continuously differences until it is. As seen above, a few of the MEVs needed one difference to achieve stationarity. The reason that we didn't difference all columns was because some already exhibited stationarity, and if we were to difference these, that could be counterproductive and introduce extra noise.

## Part 5 - Summary Statistics

### For the stock returns:

In [59]:
prices.describe()

Ticker,AAPL,AMT,AMZN,BAC,BIIB,C,CL,CVX,GOOGL,HON,...,MSFT,NEE,NKE,PFE,PG,PLD,TSN,UNP,WMT,XOM
count,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,...,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000
mean,41.089456,93.712152,45.629840,21.329719,188.913712,94.389899,45.405716,70.391148,40.853445,85.547625,...,81.555959,27.407132,48.355050,20.304765,67.874049,47.383796,34.893151,83.865282,22.885816,52.024623
std,51.693559,73.646326,53.076733,10.299661,115.398784,107.034431,19.576114,32.970924,37.465174,60.511466,...,92.311542,23.739135,41.518437,10.560747,35.353256,34.900842,23.722499,67.499705,12.447461,17.121055
min,0.583704,11.720683,1.344500,3.136746,34.450001,11.438826,13.948284,24.891178,3.235232,17.880375,...,11.903683,4.690567,7.559804,6.192620,29.262312,7.136169,5.072295,9.766551,9.826214,24.749008
25%,5.583983,32.260521,4.528000,12.113325,57.389999,35.938480,26.109665,43.791641,12.862294,32.978947,...,20.429516,9.019202,13.317992,11.159115,40.225571,24.142796,13.037644,23.320707,12.858544,39.676643
50%,19.840933,65.349609,16.122000,21.555002,219.289993,43.804787,50.561016,69.687378,26.935305,71.195633,...,34.087875,18.036430,34.348480,18.785204,58.347824,31.379827,30.307718,65.934151,19.615316,51.836784
75%,45.121761,142.240692,85.936501,29.444399,289.730011,61.727077,61.057652,85.466713,56.320042,132.702438,...,105.227173,39.103275,76.766869,27.997036,81.251450,57.604240,54.964153,134.895157,29.628235,58.165115
max,194.971756,267.634705,175.353500,44.192059,422.239990,384.071960,79.470917,169.508774,147.680313,218.677582,...,337.288818,86.963448,163.052963,51.558609,151.884415,155.346100,85.979988,257.636536,53.291927,112.671181


### The stock prices range from October of 2004 up to October of 2023 on a monthly basis.

### For the MEVs:

In [60]:
MEVs.describe()

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Mortgage rate,Prime rate,House Price Index (Level),Commercial Real Estate Price Index (Level)
count,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000,191.000000
mean,2.794764,6.005236,2.928272,-0.027225,6.140314,-0.009948,0.002094,-0.015183,-0.016230,-0.008377,0.008901,1.505759,1.560209
std,4.381165,5.247363,6.561157,10.733626,1.745249,2.392544,0.735631,0.553296,0.485905,0.494425,0.838529,2.836439,5.849056
min,-28.000000,-29.200000,-27.600000,-86.600000,3.500000,-15.200000,-3.700000,-2.200000,-2.000000,-2.200000,-4.700000,-7.800000,-37.000000
25%,1.450000,4.000000,1.250000,-2.050000,4.900000,-1.100000,-0.100000,-0.300000,-0.300000,-0.300000,-0.100000,0.500000,-0.850000
50%,2.900000,5.500000,2.900000,0.100000,5.800000,0.000000,0.000000,0.000000,0.000000,-0.100000,0.000000,1.100000,1.000000
75%,4.300000,7.650000,4.450000,2.000000,7.250000,1.200000,0.200000,0.300000,0.200000,0.200000,0.300000,2.500000,4.300000
max,34.800000,39.700000,56.000000,69.300000,13.000000,8.400000,4.500000,1.900000,1.500000,1.600000,5.100000,13.100000,23.700000


### The MEVs range from Q1 of 1976 to, similarly to stock prices, Q3 of last year. Chances are in the data analysis where the MEVs are used, the years that are in this dataset preceding those in the stock returns dataset will be dropped.

## **Section 4: Stress Testing using Fama-French three-factor model**

## Part 1 - Picking a subset of MEVs

### We're going to try and capture the big economic picture at each time step while avoiding redundancy. In order to do this, we chose to use the CPI, real GDP growth, real disposable income growth, 3 month, 5 year, and 10 year rates.

In [61]:
MEVs = MEVs[['Real GDP growth', 
             'Real disposable income growth', 
             'CPI inflation rate', 
             '3-month Treasury rate',
             '5-year Treasury yield',
             '10-year Treasury yield',
            ]]

## Part 2 - Report the results from the Fama-French three factor model

In [62]:
returns = prices.pct_change()[1:]
returns['portfolio'] = sum(returns[stock] * weights[stock] for stock, weight in weights.items()) # Setting the portfolio returns based on weights

In [63]:
FF_factors = pd.read_excel('FF_factors.xlsx')
FF_factors.set_index('Date', inplace=True)
FF_factors.head()

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,MKT,RF,SMB,HML,UMD
Date,,,,,
1963-01,0.0493,0.0025,0.0308,0.0221,-0.0211
1963-02,-0.0238,0.0023,0.0048,0.0218,0.0253
1963-03,0.0308,0.0023,-0.0259,0.0206,0.0162
1963-04,0.0451,0.0025,-0.0134,0.0100,-0.0009
1963-05,0.0176,0.0024,0.0113,0.0254,0.0033


In [64]:
FF_model = FF_factors.copy()
FF_model['MKT'] -= FF_model['RF']
FF_model.drop(columns=['RF'], inplace=True)
FF_model['portfolio'] = returns['portfolio']
FF_model = FF_model.dropna()

In [74]:
FF_portfolio = get_ols_metrics(FF_model[['MKT', 'SMB', 'HML', 'UMD']], FF_model['portfolio'])
FF_portfolio

,alpha,MKT,SMB,HML,UMD,r-squared,Info Ratio
portfolio,0.005963,0.912181,-0.271542,0.096881,-0.017913,0.901248,0.448041


## The Fama-French model for our portfolio is given by: 
$$
\Huge
E[r_i] = 0.92(MKT - r_f) - 0.3(SMB) + 0.1(HML) - 0.02(UMD) + 0.01 
$$

## Part 3 - Identify the impact of our chosen MEVs on Fama-French factors.

### Note: Here we merged the dataframes of the FF factors with the MEVs dataframe, inherently dropping all months that don't fall at the end of a quarter. This was to maintain uniformity and ensure an accurate result. We felt this method was better than interpolating MEV data because those numbers come straight from the Fed and it wouldn't be reliable to try and subjectively interpret gaps in data.

In [75]:
FF_model_MEVs = FF_factors.merge(MEVs, left_index=True, right_index=True, how='inner')

In [76]:
alltime_MEVs_effects_on_factors = pd.DataFrame(index=FF_model.columns[:-1], columns=MEVs.columns)
regressors = FF_model_MEVs[MEVs.columns]

In [77]:
for factor in FF_model.columns[:-1]:
    reg = get_ols_metrics(regressors, FF_model_MEVs[factor])
    for MEV in reg.columns[1:-2]:
        alltime_MEVs_effects_on_factors.loc[factor, MEV] = reg[MEV].values[0]

In [78]:
alltime_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
MKT,0.001038,0.000347,-0.000258,0.000558,-0.011286,-0.000289
SMB,0.000407,-0.000307,-0.000689,0.000178,-0.031251,0.031509
HML,-0.000108,0.000581,0.000561,-0.00137,0.011316,-0.011033
UMD,0.001034,-0.000257,-0.000657,-0.006656,-0.000665,-0.0067


### This dataframe shows the coefficients of the linear regressions:
$$
\Huge
FF_i = \alpha_i + \beta_{1_i}MEV_1 + \beta_2MEV_2 + \ldots + \beta_nMEV_n
$$

## Part 4 - Investigating the impact of the MEVs on Fama-French factors during *stressed times*

### We are picking all date ranges of stressed times after 1976, where our data for FF factors and MEVs starts. This gives a full picture on the effect of the MEVs on the FF factors throughout history of the past ~5 decades, whereas a smaller sample size wouldn't accurately describe the effects.

In [79]:
df = FF_factors.merge(MEVs, left_index=True, right_index=True, how='inner')
stressed_date_ranges = [('1980-01', '1980-06'),
                        ('1981-06', '1982-12'),
                        ('1990-06', '1991-03'),
                        ('2001-03', '2001-12'),
                        ('2007-12', '2009-06'),
                        ('2020-03', '2020-06')
                       ]
stressed_data = pd.concat(
    [df.loc[start:end] for start, end in stressed_date_ranges]
)

In [80]:
stressed_MEVs_effects_on_factors = pd.DataFrame(index=FF_model.columns[:-1], columns=MEVs.columns)
regressors = stressed_data[MEVs.columns]
for factor in FF_model.columns[:-1]:
    reg = get_ols_metrics(regressors, stressed_data[factor])
    for MEV in reg.columns[1:-2]:
        stressed_MEVs_effects_on_factors.loc[factor, MEV] = reg[MEV].values[0]
stressed_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
MKT,-0.003533,-0.001477,0.000422,-0.005826,0.021461,-0.043141
SMB,-0.000399,-0.000226,-0.001747,-0.016149,0.017437,-0.008091
HML,0.000063,-0.000588,0.000285,0.00031,0.007701,-0.002738
UMD,0.006836,0.003736,0.001189,-0.007574,-0.022251,0.003064


### This table similarly shows the coefficients of the MEVs on FF factors.

## Part 5 - Projecting the performance of our portfolio

In [88]:
# Read in the adverse situation MEVs
MEVs_severe = pd.read_csv('MEV_severe.csv')
MEVs_severe['Date'] = MEVs_severe['Date'].apply(convert_to_yyyy_mm)
MEVs_severe.set_index('Date', inplace=True)
MEVs_severe = MEVs_severe[MEVs.columns] # Filter to only the MEVs we're using

In [89]:
# Handle stationarity similar to before
diffs_needed = pd.DataFrame(index=MEVs_severe.columns, columns=['Differences'])
for MEV in MEVs_severe.columns:
    stationary = make_stationary(MEVs_severe[MEV])
    diffs_needed.loc[MEV] = stationary[0]
    if stationary[0] != 0:
        MEVs_severe[MEV] = stationary[1]
MEVs_severe = MEVs_severe.dropna()

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:958: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
Date,,,,,,
1976-06,3.0,2.3,-1.1,0.3,0.0,0.0
1976-09,2.2,3.2,2.9,0.0,-0.1,0.0
1976-12,2.9,2.6,-0.6,-0.5,-0.8,-0.5
1977-03,4.8,0.9,1.6,-0.1,0.3,0.1
1977-06,8.0,3.8,-0.3,0.2,0.0,0.1


In [91]:
MEVs_severe.head()

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
Date,,,,,,
2025-03,2.0,2.8,0.1,0.1,0.0,4.000000e-01
2025-06,-1.7,1.3,0.0,0.1,0.1,-2.000000e-01
2025-09,-2.1,0.5,0.0,0.1,0.1,1.000000e-01
2025-12,5.3,3.6,0.1,0.1,0.0,-4.440892e-16
2026-03,-6.0,0.3,0.0,0.1,0.1,7.771561e-16


In [90]:
alltime_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
MKT,0.001038,0.000347,-0.000258,0.000558,-0.011286,-0.000289
SMB,0.000407,-0.000307,-0.000689,0.000178,-0.031251,0.031509
HML,-0.000108,0.000581,0.000561,-0.00137,0.011316,-0.011033
UMD,0.001034,-0.000257,-0.000657,-0.006656,-0.000665,-0.0067


In [ ]:
# TODO: run the regression using all time MEV coefficients against severe MEV values to get MKT, SMB, HML, UMD values 
# and then plug those values into the regression equation for earlier at each time step to get portfolio return